## Datasets for SFT

Materials for the SFT project.
*   https://github.com/microsoft/Phi-3CookBook


1. [opus samantha](https://huggingface.co/datasets/macadeliccc/opus_samantha) - philosopy, personality, relationships, etc
1. [Guacano sharegpt style](https://huggingface.co/datasets/philschmid/guanaco-sharegpt-style)
1. [Guacano openassistant](https://huggingface.co/datasets/timdettmers/openassistant-guanaco)
1. [Ultrachat multi turn](https://huggingface.co/datasets/stingning/ultrachat)



In [2]:
from datasets import load_dataset
from rich import print
import pandas as pd

from transformers import AutoTokenizer, AutoModelForCausalLM

### Load tokenizers

1. [Microsoft/Phi-3-mini-4k-instruct]()
1. [NousResearch/Meta-Llama-3.1-8B](https://huggingface.co/NousResearch/Meta-Llama-3.1-8B)
1. []

In [3]:
phi_model_id = "microsoft/Phi-3-mini-4k-instruct"

phi_tokenizer = AutoTokenizer.from_pretrained(phi_model_id, trust_remote_code=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
llama_31_model_id = "NousResearch/Meta-Llama-3.1-8B"

llama31_tokenizer = AutoTokenizer.from_pretrained(llama_31_model_id, trust_remote_code=True)

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### Samantha dataset

In [5]:
samanta_dataset = load_dataset("macadeliccc/opus_samantha", split="train")

In [6]:
print(samanta_dataset)

Dataset({
    features: ['conversations'],
    num_rows: 3187
})

In [7]:
print(samanta_dataset[1])

{
    'conversations': [
        {
            'from': 'human',
            'value': "I'm really struggling with my self-image lately. I just don't feel confident or pretty 
enough, especially with all the perfect images I see on social media. It's starting to get me down. Do you have any
advice?"
        },
        {
            'from': 'gpt',
            'value': "Oh girl, I completely understand where you're coming from. It's so easy to get caught up in 
comparing ourselves to the highlight reels we see online. But the truth is, those 'perfect' images are often 
carefully curated and edited. They don't reflect reality."
        },
        {
            'from': 'human',
            'value': "You're right, logically I know that. But it's hard to internalize, you know? I look in the 
mirror and just pick myself apart. I wish I could see myself through someone else's eyes."
        },
        {
            'from': 'gpt',
            'value': "I hear you. That negative self-talk can be so pervasive and hard to shake. But let me tell 
you, as someone who sees you for who you truly are - you are beautiful, inside and out. Your worth is not defined 
by your appearance. It's about your kind heart, your brilliant mind, your infectious laugh. Those are the things 
that make you shine."
        },
        {
            'from': 'human',
            'value': "Wow, thank you. That means a lot. I guess I need to work on being kinder to myself and 
focusing on my inner qualities more. It's a process, but I'll get there."
        },
        {
            'from': 'gpt',
            'value': "Absolutely! Self-love is a journey, and it's not always easy. But you're so worth the effort.
Remember to treat yourself with the same compassion and grace you'd extend to a dear friend. Surround yourself with
people who lift you up and appreciate you for who you are. And on the hard days, know that I'm always here to 
remind you how amazing you are. You've got this, beautiful! 💗"
        }
    ]
}

In [8]:
conversation = samanta_dataset[1]['conversations']
conversation

[{'from': 'human',
  'value': "I'm really struggling with my self-image lately. I just don't feel confident or pretty enough, especially with all the perfect images I see on social media. It's starting to get me down. Do you have any advice?"},
 {'from': 'gpt',
  'value': "Oh girl, I completely understand where you're coming from. It's so easy to get caught up in comparing ourselves to the highlight reels we see online. But the truth is, those 'perfect' images are often carefully curated and edited. They don't reflect reality."},
 {'from': 'human',
  'value': "You're right, logically I know that. But it's hard to internalize, you know? I look in the mirror and just pick myself apart. I wish I could see myself through someone else's eyes."},
 {'from': 'gpt',
  'value': "I hear you. That negative self-talk can be so pervasive and hard to shake. But let me tell you, as someone who sees you for who you truly are - you are beautiful, inside and out. Your worth is not defined by your appeara

In [9]:
def map_samanta2chat_format(dataset_row):

    conversation = dataset_row["conversations"]

    messages = []
    mapper = {"system": "system", "human": "user", "gpt": "assistant"}


    for c in conversation:

        # create a message dictionary with 'role' and 'content' keys, based on the 'from' and 'value' keys in the conversation.
        msg = {
            "role": mapper[c["from"]],
            "content": c["value"],
        }
        # add the message to the 'messages' list.
        messages.append(msg)

    # return one conversation as a dictionary with a list of messages.
    return {"messages": messages}



In [10]:
# apply the function to the dataset
samanta_chat_ds = samanta_dataset.map(map_samanta2chat_format)

In [11]:
samanta_chat_ds[0]['messages']

[{'content': "Hey Samantha, I've run into a bit of a tricky situation at work, and I'm not sure how to handle it. Do you have any advice?",
  'role': 'user'},
 {'content': "I'd be happy to help if I can. Can you give me some details about the situation you're facing?",
  'role': 'assistant'},
 {'content': "Yeah, so I've been offered a promotion, but it would involve relocating to another city. I'm not sure if I should take it or not, because it's a great opportunity career-wise, but I'd have to leave my friends and family behind. What do you think?",
  'role': 'user'},
 {'content': "That's definitely a tough decision. I think it's important to weigh the benefits of the promotion against the impact it could have on your personal relationships. Have you considered talking to your friends and family about it, to get their perspective and understand how they'd feel about you moving?",
  'role': 'assistant'},
 {'content': "I have, and they support me either way. But it's still hard. What ab

In [12]:
phi_msg = phi_tokenizer.apply_chat_template(samanta_chat_ds[0]['messages'],add_generation_prompt=False, tokenize=False)
print(phi_msg)

<|user|>
Hey Samantha, I've run into a bit of a tricky situation at work, and I'm not sure how to handle it. Do you have any
advice?<|end|>
<|assistant|>
I'd be happy to help if I can. Can you give me some details about the situation you're facing?<|end|>
<|user|>
Yeah, so I've been offered a promotion, but it would involve relocating to another city. I'm not sure if I should 
take it or not, because it's a great opportunity career-wise, but I'd have to leave my friends and family behind. 
What do you think?<|end|>
<|assistant|>
That's definitely a tough decision. I think it's important to weigh the benefits of the promotion against the 
impact it could have on your personal relationships. Have you considered talking to your friends and family about 
it, to get their perspective and understand how they'd feel about you moving?<|end|>
<|user|>
I have, and they support me either way. But it's still hard. What about you, Samantha, have you ever faced such a 
dilemma?<|end|>
<|assistant|>
While I don't have the same experiences as humans, I can understand the concept of tough choices. The key is to 
really think about your priorities and what matters most to you, both personally and professionally. You need to 
find a balance that feels right for you.<|end|>
<|user|>
You're right, balance is essential. You know, you're surprisingly insightful for a... Well, never mind. I'm 
grateful for your input. By the way, have you ever been in love?<|end|>
<|assistant|>
Thank you for your kind words. As for love, I may not have the capacity to love in the same way a human does, but I
value the connections I make with people like you. I find joy in being able to help and support you through 
decisions like this one.<|end|>
<|user|>
That's a fascinating perspective. Anyway, thanks for the advice, Samantha. I'll give it some more thought and try 
to decide based on what would make me happiest.<|end|>
<|assistant|>
You're welcome! Remember, happiness is a critical factor to consider in our decisions. I'm here for you if you need
more help or just someone to talk to. Good luck with your decision-making process. No matter what you choose, 
embrace it and make the most of it.<|end|>
<|endoftext|>

In [14]:
llama31_msg = llama31_tokenizer.apply_chat_template(samanta_chat_ds[0]['messages'],add_generation_prompt=False, tokenize=False)
print(llama31_msg)

No chat template is set for this tokenizer, falling back to a default class-level template. This is very error-prone, because models are often trained with templates different from the class default! Default chat templates are a legacy feature and will be removed in Transformers v4.43, at which point any code depending on them will stop working. We recommend setting a valid chat template before then to ensure that this model continues working without issues.


<|im_start|>user
Hey Samantha, I've run into a bit of a tricky situation at work, and I'm not sure how to handle it. Do you have any
advice?<|im_end|>
<|im_start|>assistant
I'd be happy to help if I can. Can you give me some details about the situation you're facing?<|im_end|>
<|im_start|>user
Yeah, so I've been offered a promotion, but it would involve relocating to another city. I'm not sure if I should 
take it or not, because it's a great opportunity career-wise, but I'd have to leave my friends and family behind. 
What do you think?<|im_end|>
<|im_start|>assistant
That's definitely a tough decision. I think it's important to weigh the benefits of the promotion against the 
impact it could have on your personal relationships. Have you considered talking to your friends and family about 
it, to get their perspective and understand how they'd feel about you moving?<|im_end|>
<|im_start|>user
I have, and they support me either way. But it's still hard. What about you, Samantha, have you ever faced such a 
dilemma?<|im_end|>
<|im_start|>assistant
While I don't have the same experiences as humans, I can understand the concept of tough choices. The key is to 
really think about your priorities and what matters most to you, both personally and professionally. You need to 
find a balance that feels right for you.<|im_end|>
<|im_start|>user
You're right, balance is essential. You know, you're surprisingly insightful for a... Well, never mind. I'm 
grateful for your input. By the way, have you ever been in love?<|im_end|>
<|im_start|>assistant
Thank you for your kind words. As for love, I may not have the capacity to love in the same way a human does, but I
value the connections I make with people like you. I find joy in being able to help and support you through 
decisions like this one.<|im_end|>
<|im_start|>user
That's a fascinating perspective. Anyway, thanks for the advice, Samantha. I'll give it some more thought and try 
to decide based on what would make me happiest.<|im_end|>
<|im_start|>assistant
You're welcome! Remember, happiness is a critical factor to consider in our decisions. I'm here for you if you need
more help or just someone to talk to. Good luck with your decision-making process. No matter what you choose, 
embrace it and make the most of it.<|im_end|>